# I don't have time to Manually do it
## So here comes built-ins

In [0]:
import numpy as np
import torch
import torch.nn as nn

# Training Data
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], 
                   [102, 43, 37], [69, 96, 70], [73, 67, 43], 
                   [91, 88, 64], [87, 134, 58], [102, 43, 37], 
                   [69, 96, 70], [73, 67, 43], [91, 88, 64], 
                   [87, 134, 58], [102, 43, 37], [69, 96, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133], 
                    [22, 37], [103, 119], [56, 70], 
                    [81, 101], [119, 133], [22, 37], 
                    [103, 119], [56, 70], [81, 101], 
                    [119, 133], [22, 37], [103, 119]], 
                   dtype='float32')

# Converting numpy arrays to pytorch tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

## Datasets and DataLoader

### TensorDataset
We will create a TensorDataset. It can access rows of inputs and targets as tuples; provides standard API 

In [3]:
from torch.utils.data import TensorDataset

# define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3] # I want a slice

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

Returns a tuple of size 2. First element is the first 3 inputs, second element is the first 3 targets

### Dataloader
Can spllit data into batches. Also shuffles, random samplings etc

In [4]:
from torch.utils.data import DataLoader

# define DataLoader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

# Dataloader is typically used in a for loop
for x_batch, y_batch in train_dl: # takes 5 samples as a batch
    print(x_batch)
    print(y_batch)
    break

tensor([[ 69.,  96.,  70.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [ 69.,  96.,  70.],
        [ 73.,  67.,  43.]])
tensor([[103., 119.],
        [ 81., 101.],
        [119., 133.],
        [103., 119.],
        [ 56.,  70.]])


## nn.Linear

Instead of initializing the weights & biases manually, we can define the model using the nn.Linear class from PyTorch, which does it automatically.

In [5]:
# define model
# Applies a linear transformation to the incoming data: :math:y = xA^T + b
model = nn.Linear(in_features=3, out_features=2, bias=True)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.3928,  0.1208, -0.0894],
        [-0.5004,  0.5238, -0.5435]], requires_grad=True)
Parameter containing:
tensor([-0.0960,  0.5425], requires_grad=True)


In [6]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.3928,  0.1208, -0.0894],
         [-0.5004,  0.5238, -0.5435]], requires_grad=True),
 Parameter containing:
 tensor([-0.0960,  0.5425], requires_grad=True)]

In [7]:
# Prediction is same as before
preds = model(inputs); preds

tensor([[ 32.8236, -24.2682],
        [ 40.5518, -33.6910],
        [ 45.0728,  -4.3348],
        [ 41.8526, -48.0898],
        [ 32.3400, -21.7528],
        [ 32.8236, -24.2682],
        [ 40.5518, -33.6910],
        [ 45.0728,  -4.3348],
        [ 41.8526, -48.0898],
        [ 32.3400, -21.7528],
        [ 32.8236, -24.2682],
        [ 40.5518, -33.6910],
        [ 45.0728,  -4.3348],
        [ 41.8526, -48.0898],
        [ 32.3400, -21.7528]], grad_fn=<AddmmBackward>)

## Loss function built in

In [0]:
import torch.nn.functional as F
loss_fn = F.mse_loss

In [12]:
# Compute loss
loss = loss_fn(model(inputs), targets)
initial_loss = loss
print(initial_loss)
print(loss.item())      

tensor(8596.6006, grad_fn=<MseLossBackward>)
8596.6005859375


## Optimizer

In [0]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr = 1e-5)

## Model Training


Using data loader for extracting data batches

opt.step is to update the parameters

opt.zero_grad is to reset gradients

loss.item returns actual value stored in loss tensor

In [0]:
def fit(train_dl, model, loss_fn, opt, num_epochs):
    for epoch in range(num_epochs):
        # get batches in for loop
        for x_batch, y_batch in train_dl:
            # predict
            pred = model(x_batch)
            # loss
            loss = loss_fn(pred, y_batch)
            # Compute Gradients
            loss.backward()
            # Update parameters using gradients, without calculating gradients further (no_grad())
            opt.step()
            # Reset gradients to zero
            opt.zero_grad()
        
        # verbose
        if (epoch+1) % 10 == 0:
            print('Epoch {}/{}, loss = {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [14]:
fit(train_dl=train_dl, model=model, loss_fn=loss_fn, opt=opt, num_epochs=100)

Epoch 10/100, loss = 313.1768
Epoch 20/100, loss = 128.8890
Epoch 30/100, loss = 408.1381
Epoch 40/100, loss = 131.3058
Epoch 50/100, loss = 132.9145
Epoch 60/100, loss = 135.2191
Epoch 70/100, loss = 69.6358
Epoch 80/100, loss = 30.4960
Epoch 90/100, loss = 77.5364
Epoch 100/100, loss = 72.6379


## Generating Predictions

In [15]:
preds = model(inputs); preds

tensor([[ 58.7713,  71.1892],
        [ 79.9734,  94.5649],
        [120.8813, 144.8109],
        [ 30.4236,  41.4137],
        [ 92.5861, 106.1031],
        [ 58.7713,  71.1892],
        [ 79.9734,  94.5649],
        [120.8813, 144.8109],
        [ 30.4236,  41.4137],
        [ 92.5861, 106.1031],
        [ 58.7713,  71.1892],
        [ 79.9734,  94.5649],
        [120.8813, 144.8109],
        [ 30.4236,  41.4137],
        [ 92.5861, 106.1031]], grad_fn=<AddmmBackward>)